In [19]:
import geopandas as gpd 
import math
from shapely.geometry import LineString

In [20]:
MHWL = gpd.read_file('C:/Users/lalit/GitHub/Shoreline_Projections/Data/nz-MHWL.shp')
transect_rates = gpd.read_file('C:/Users/lalit/GitHub/Shoreline_Projections/Data/TorrentBay_rates.shp')

In [21]:
extended_transects = gpd.GeoDataFrame()

In [22]:
def get_azimuth(line):
    p1, p2 = line.coords[1:]
    azimuth = math.degrees(math.atan2(p2[0]-p1[0], p2[1]-p1[1]))
    if azimuth < 0:
        azimuth += 360
    return azimuth
transect_rates["Azimuth"] = transect_rates.geometry.apply(get_azimuth)
transect_rates.to_file("Data/TorrentBay_rates.shp")
transect_rates

,ObjectID_1,BaselineID,TransOrder,TransEdit,transectId,ShrCount,LRR,LR2,LCI,LSE,...,EPRunc,NSM,SCE,WLR,WR2,WCI,WSE,TCD,Azimuth,geometry
0,1.0,0.0,1.0,0.0,1.0,10.0,0.02,0.04,0.05,2.29,...,0.06,1.29,7.11,0.00,0.00,0.05,0.67,0.00,239.162167,"LINESTRING Z (1605478.052 5466338.300 0.000, 1..."
1,2.0,0.0,2.0,0.0,2.0,10.0,0.03,0.12,0.04,2.07,...,0.06,1.57,6.88,0.01,0.03,0.05,0.63,10.00,239.162167,"LINESTRING Z (1605472.926 5466346.886 0.000, 1..."
2,3.0,0.0,3.0,0.0,3.0,10.0,0.02,0.14,0.04,1.87,...,0.06,1.70,6.24,0.01,0.04,0.05,0.60,20.00,239.162167,"LINESTRING Z (1605467.800 5466355.472 0.000, 1..."
3,4.0,0.0,4.0,0.0,4.0,10.0,0.02,0.15,0.04,1.78,...,0.06,2.18,5.27,0.02,0.06,0.05,0.61,30.00,239.162169,"LINESTRING Z (1605462.673 5466364.058 0.000, 1..."
4,5.0,0.0,5.0,0.0,5.0,10.0,0.02,0.14,0.04,1.75,...,0.06,2.43,5.28,0.02,0.07,0.04,0.56,40.00,239.162168,"LINESTRING Z (1605457.547 5466372.644 0.000, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,466.0,12.0,466.0,0.0,466.0,8.0,-0.21,0.65,0.12,3.36,...,0.10,-11.14,17.03,-0.21,0.55,0.15,1.10,12140.35,285.851929,"LINESTRING Z (1604435.973 5474504.870 0.000, 1..."
463,467.0,12.0,467.0,0.0,467.0,8.0,-0.19,0.56,0.13,3.56,...,0.10,-10.18,16.46,-0.19,0.43,0.17,1.23,12150.35,285.851927,"LINESTRING Z (1604438.704 5474514.490 0.000, 1..."
464,468.0,12.0,468.0,0.0,468.0,8.0,-0.15,0.54,0.11,3.00,...,0.10,-8.83,12.82,-0.15,0.44,0.13,0.97,12160.35,287.241459,"LINESTRING Z (1604441.654 5474524.045 0.000, 1..."
465,469.0,12.0,469.0,0.0,469.0,8.0,0.02,0.03,0.09,2.40,...,0.10,2.35,6.11,0.02,0.02,0.13,0.92,12170.35,324.462323,"LINESTRING Z (1604447.621 5474531.425 0.000, 1..."


In [24]:
for index, transect in transect_rates.iterrows():
    if transect['geometry'].intersects(MHWL.unary_union):
        extended_transects = extended_transects.append({'transect_id':transect['transectId'], 
                                                        'intersect_point': None, 
                                                        'geometry': transect['geometry']}, 
                                                        ignore_index=True)
    else:
        intersect_point = transect ['geometry'].intersection(MHWL.unary_union)
        extended_line = LineString([transect['geometry'].coords[0], intersect_point])
        extended_transects = extended_transects.append({'transect_id': transect['transectId'], 
                                                        'intersect_point': intersect_point, 
                                                        'geometry': extended_line}, 
                                                        ignore_index=True)

AttributeError: 'GeoDataFrame' object has no attribute 'append'

In [ ]:
extended_transects.to_file('Data/extended_transects.geojson')